# Préparation des variables de mode de vie

In [2]:
library(tidyverse)
library(factoextra)

# Charger les données
DataApp <- readRDS("../../_PrivateFolder_datagotchi_federal_2025/data/clustering/can2025/01_app_2025.rds")
DensityData <- readRDS("../../_SharedFolder_datagotchi_federal_2024/data/clustering/can_2025/density_data.rds")

# Charger le fichier des variables
source("01-1_variables.R")

df_mode_de_vie <- DataApp %>%
  select(all_of(variables_mode_de_vie))

In [3]:
library(dplyr)

df_mode_de_vie %>%
  select(starts_with("lifestyle_typeTransport")) %>%
  summarise(across(everything(), \(x) sum(x, na.rm = TRUE)))
#  summarise(across(everything(), sum, na.rm = TRUE))

lifestyle_typeTransportCar,lifestyle_typeTransportSUV,lifestyle_typeTransportMoto,lifestyle_typeTransportWalk,lifestyle_typeTransportBicycle,lifestyle_typeTransportPublicTransit
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
28683,4903,371,3897,943,4279


In [4]:
# Boucle pour afficher la table pour chaque variable
for (v in variables_mode_de_vie) {
  if (v != "id") {
    cat("Table pour la variable:", v, "\n")
    print(table(df_mode_de_vie[[v]]))
    cat("\n\n")
  }
}

Table pour la variable: ses_postalCode 

A0A A0B A0C A0E A0G A0H A0J A0K A0L A0M A0N A0P A1A A1B A1C A1E A1G A1H A1K A1L 
 19   5  11   8  12   9   2   5   2   2   6   3  33  14  15  22   5   4   9  10 
A1M A1N A1S A1V A1W A1X A1Y A2A A2B A2H A2N A5A A7S A8A B0B B0E B0H B0J B0K B0L 
  3  14   2   9   5   9   3   2   4  19   6   4   1   5   1   8   3  21  17   5 
B0M B0N B0P B0R B0S B0T B0V B0W B0X B1A B1E B1G B1H B1K B1L B1M B1N B1P B1R B1S 
  6  28  21   3  18   5   6  14   1   7   1   1   6   2   1   1   4   8   2   3 
B1T B1V B1W B1Y B2A B2C B2G B2H B2K B2N B2R B2S B2T B2V B2W B2X B2Y B2Z B3A B3B 
  1   5   1   2   6   1   7  14   1  13   2   3  12   8  23   9  11   3  13   7 
B3E B3G B3H B3J B3K B3L B3M B3N B3P B3R B3S B3T B3V B3Z B4A B4B B4C B4E B4G B4H 
 11   8  27   9  25   7  42   8   4   7  17  10   2  11  16  18  13  16   2   4 
B4N B4P B4V B5A B5G B6L B7J B8L B9A B9N C0A C0B C0J C1A C1B C1C C1E C1N C3C C3X 
  9   7  10   7   1  11   1   1   1   1  23  17   1  27   7   5   6 

### Transformations

- Regroupement de ses_dwelling
  - ses_dwellingCondo : ses_dwellingCondo + ses_dwellingLoft + ses_dwellingTour
  - ses_dwellingOther : ses_dwellingOther + ses_dwellingCoop + ses_dwellingHLM + ses_dwellingMobile

- Regroupement des act_transport
  - lifestyle_typeTransportCar : lifestyle_typeTransportCar + lifestyle_typeTransportMoto (parce qu'il y avraiment trop peu de moto, assigner la classe la plus populeuse)
  - lifestyle_typeTransportActive:   lifestyle_typeTransportWalk + lifestyle_typeTransportBicycle
  - Les autres ne sont pas regroupés

In [5]:
# Regroupement de ses_dwelling
#-----------------------------

df_mode_de_vie <- df_mode_de_vie %>%

  mutate(
    ses_dwellingCondo = if_else(
      ses_dwellingLoft == 1 | ses_dwellingTour == 1,
      1,
      ses_dwellingCondo
    ),
    ses_dwellingOther = if_else(
      ses_dwellingCoop == 1 | ses_dwellingHLM == 1 | ses_dwellingMobile == 1,
      1,
      ses_dwellingOther
    )
  ) %>%
  select(
    -ses_dwellingLoft,
    -ses_dwellingTour,
    -ses_dwellingCoop,
    -ses_dwellingHLM,
    -ses_dwellingMobile
  )

In [6]:
# Regroupement de act_transport
#------------------------------

df_mode_de_vie <- df_mode_de_vie %>%
  mutate(
    lifestyle_typeTransportCar = if_else(
      lifestyle_typeTransportMoto == 1 | lifestyle_typeTransportSUV == 1,
      1,
      lifestyle_typeTransportCar
    ),
    lifestyle_typeTransportNoCar = if_else(
      lifestyle_typeTransportWalk == 1 | lifestyle_typeTransportBicycle == 1 | lifestyle_typeTransportPublicTransit == 1,
      1,
      0
    )
  ) %>%
  select(
    -lifestyle_typeTransportMoto,
    -lifestyle_typeTransportSUV,
    -lifestyle_typeTransportWalk,
    -lifestyle_typeTransportBicycle,
    -lifestyle_typeTransportPublicTransit)

In [7]:
variables_mode_de_vie_clust <- c(
  "id",
  "ses_postalCode",
  "ses_dwellingApp",
  "ses_dwellingCondo",
  "ses_dwellingDetachedHouse",
  "ses_dwellingTownhouse",
  "ses_dwellingDuplex",
  "ses_dwellingOther",
  "lifestyle_typeTransportCar",
  "lifestyle_typeTransportNoCar"
)

In [8]:
# Boucle pour afficher la table pour chaque variable
for (v in variables_mode_de_vie_clust) {
  if (v != "id") {
    cat("Table pour la variable:", v, "\n")
    print(table(df_mode_de_vie[[v]]))
    cat("\n\n")
  }
}

Table pour la variable: ses_postalCode 

A0A A0B A0C A0E A0G A0H A0J A0K A0L A0M A0N A0P A1A A1B A1C A1E A1G A1H A1K A1L 
 19   5  11   8  12   9   2   5   2   2   6   3  33  14  15  22   5   4   9  10 
A1M A1N A1S A1V A1W A1X A1Y A2A A2B A2H A2N A5A A7S A8A B0B B0E B0H B0J B0K B0L 
  3  14   2   9   5   9   3   2   4  19   6   4   1   5   1   8   3  21  17   5 
B0M B0N B0P B0R B0S B0T B0V B0W B0X B1A B1E B1G B1H B1K B1L B1M B1N B1P B1R B1S 
  6  28  21   3  18   5   6  14   1   7   1   1   6   2   1   1   4   8   2   3 
B1T B1V B1W B1Y B2A B2C B2G B2H B2K B2N B2R B2S B2T B2V B2W B2X B2Y B2Z B3A B3B 
  1   5   1   2   6   1   7  14   1  13   2   3  12   8  23   9  11   3  13   7 
B3E B3G B3H B3J B3K B3L B3M B3N B3P B3R B3S B3T B3V B3Z B4A B4B B4C B4E B4G B4H 
 11   8  27   9  25   7  42   8   4   7  17  10   2  11  16  18  13  16   2   4 
B4N B4P B4V B5A B5G B6L B7J B8L B9A B9N C0A C0B C0J C1A C1B C1C C1E C1N C3C C3X 
  9   7  10   7   1  11   1   1   1   1  23  17   1  27   7   5   6 

In [9]:
library(tidyverse)

# 1. Mettre tous les codes postaux en majuscule
df_mode_de_vie <- df_mode_de_vie %>%
  mutate(ses_postalCode = toupper(ses_postalCode))

# 2. Première jointure pour récupérer la classification
df_mode_de_vie <- left_join(df_mode_de_vie, DensityData, by = c("ses_postalCode" = "fsa")) %>%
  rename(ses_urbanity = ses_population_centre)

# 3. Créer une variable avec la première lettre du code postal
df_mode_de_vie <- df_mode_de_vie %>%
  mutate(initial = substr(ses_postalCode, 1, 1))

# 4. Calculer, pour chaque lettre, le code postal (FSA) le plus populaire parmi les observations valides
mode_fsa_by_letter <- df_mode_de_vie %>%
  filter(!is.na(ses_urbanity)) %>%
  group_by(initial) %>%
  count(ses_postalCode, sort = TRUE) %>%
  slice(1) %>%      # Garde le code postal le plus fréquent pour chaque initiale
  ungroup() %>%
  select(initial, mode_postal = ses_postalCode)

# 5. Imputer le mode par initial pour les codes non appariés (où ses_urbanity est NA)
df_mode_de_vie <- df_mode_de_vie %>%
  left_join(mode_fsa_by_letter, by = "initial") %>%
  mutate(ses_postalCode = if_else(is.na(ses_urbanity), mode_postal, ses_postalCode)) %>%
  select(-mode_postal, -initial)

# 6. Supprimer l'ancienne variable de classification
df_mode_de_vie <- df_mode_de_vie %>% select(-ses_urbanity)

# 7. Refaire la jointure avec DensityData pour mettre à jour la classification avec les codes imputés
df_mode_de_vie <- left_join(df_mode_de_vie, DensityData, by = c("ses_postalCode" = "fsa")) %>%
  rename(ses_urbanity = ses_population_centre)

# Vérifier le résultat de la jointure
table(df_mode_de_vie$ses_urbanity, useNA = "ifany")

# 8. Créer la variable binaire : urbain (1) si "population_centre", sinon rural (0)
df_mode_de_vie <- df_mode_de_vie %>%
  mutate(ses_urban = if_else(ses_urbanity == "population_centre", 1, 0))

table(df_mode_de_vie$ses_urban, useNA = "ifany")



population_centre             rural 
            25892             17184 


    0     1 
17184 25892 

In [10]:

names(df_mode_de_vie)
# Sauvegarder les données préparées
saveRDS(df_mode_de_vie, file = "../../_PrivateFolder_datagotchi_federal_2025/data/clustering/can2025/02_app_2025_mode_de_vie.rds")

[1] "id"                           "ses_postalCode"              
 [3] "ses_dwellingApp"              "ses_dwellingCondo"           
 [5] "ses_dwellingDetachedHouse"    "ses_dwellingTownhouse"       
 [7] "ses_dwellingDuplex"           "ses_dwellingOther"           
 [9] "lifestyle_typeTransportCar"   "lifestyle_typeTransportNoCar"
[11] "population.x"                 "area_km2.x"                  
[13] "density_by_km2.x"             "province.x"                  
[15] "population.y"                 "area_km2.y"                  
[17] "density_by_km2.y"             "province.y"                  
[19] "ses_urbanity"                 "ses_urban"

---

# SUGGESTION : Activité de Clustering

Si souhaité, effectuer un exercice de clustering avec ce sous-groupe de variables pour mieux saisir les données et leur relation. 

Cet exercice est purement exploratoire et sert à voir si les données peuvent être utilisées pour faire du clustering.